In [1]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Use BigQuery DataFrames to cluster and characterize complaints

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/googleapis/python-bigquery-dataframes/blob/main/notebooks/generative_ai/bq_dataframes_llm_kmeans.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/googleapis/python-bigquery-dataframes/tree/main/notebooks/generative_ai/bq_dataframes_llm_kmeans.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/googleapis/python-bigquery-dataframes/tree/main/notebooks/generative_ai/bq_dataframes_llm_kmeans.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

## Overview

The goal of this notebook is to demonstrate a comment characterization algorithm for an online business. We will accomplish this using [Google's PaLM 2](https://ai.google/discover/palm2/) and [KMeans clustering](https://en.wikipedia.org/wiki/K-means_clustering) in three steps:

1. Use PaLM2TextEmbeddingGenerator to [generate text embeddings](https://cloud.google.com/vertex-ai/docs/generative-ai/embeddings/get-text-embeddings) for each of 10000 complaints sent to an online bank. If you're not familiar with what a text embedding is, it's a list of numbers that are like coordinates in an imaginary "meaning space" for sentences. (It's like [word embeddings](https://en.wikipedia.org/wiki/Word_embedding), but for more general text.) The important point for our purposes is that similar sentences are close to each other in this imaginary space.
2. Use KMeans clustering to group together complaints whose text embeddings are near to eachother. This will give us sets of similar complaints, but we don't yet know _why_ these complaints are similar.
3. Prompt GeminiTextGenerator in English asking what the difference is between the groups of complaints that we got. Thanks to the power of modern LLMs, the response might give us a very good idea of what these complaints are all about, but remember to ["understand the limits of your dataset and model."](https://ai.google/responsibility/responsible-ai-practices/#:~:text=Understand%20the%20limitations%20of%20your%20dataset%20and%20model)

We will tie these pieces together in Python using BigQuery DataFrames. [Click here](https://cloud.google.com/bigquery/docs/dataframes-quickstart) to learn more about BigQuery DataFrames!

### Dataset

This notebook uses the [CFPB Consumer Complaint Database](https://console.cloud.google.com/marketplace/product/cfpb/complaint-database).

### Costs

This tutorial uses billable components of Google Cloud:

* BigQuery (compute)
* BigQuery ML
* Generative AI support on Vertex AI

Learn about [BigQuery compute pricing](https://cloud.google.com/bigquery/pricing#analysis_pricing_models), [Generative AI support on Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing#generative_ai_models),
and [BigQuery ML pricing](https://cloud.google.com/bigquery/pricing#bqml),
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Before you begin

Complete the tasks in this section to set up your environment.

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

3. [Click here](https://console.cloud.google.com/flows/enableapi?apiid=bigquery.googleapis.com,bigqueryconnection.googleapis.com,aiplatform.googleapis.com) to enable the following APIs:

  * BigQuery API
  * BigQuery Connection API
  * Vertex AI API

4. If you are running this notebook locally, install the [Cloud SDK](https://cloud.google.com/sdk).

#### Set your project ID

**If you don't know your project ID**, see the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [2]:
# set your project ID below
PROJECT_ID = ""  # @param {type:"string"}

# Set the project id in gcloud
#! gcloud config set project {PROJECT_ID}

#### Authenticate your Google Cloud account

Depending on your Jupyter environment, you might have to manually authenticate. Follow the relevant instructions below.

**Vertex AI Workbench**

Do nothing, you are already authenticated.

**Local JupyterLab instance**

Uncomment and run the following cell:

In [3]:
# ! gcloud auth login

**Colab**

Uncomment and run the following cell:

In [4]:
# from google.colab import auth
# auth.authenticate_user()

Now we are ready to use BigQuery DataFrames!

## Step 1: Text embedding 

BigQuery DataFrames setup

In [5]:
import bigframes.pandas as bf

# Note: The project option is not required in all environments.
# On BigQuery Studio, the project ID is automatically detected.
bf.options.bigquery.project = PROJECT_ID

If you want to reset the location of the created DataFrame or Series objects, reset the session by executing `bf.close_session()`. After that, you can reuse `bf.options.bigquery.location` to specify another location.

Data Input - read the data from a publicly available BigQuery dataset

In [6]:
input_df = bf.read_gbq("bigquery-public-data.cfpb_complaints.complaint_database")

In [7]:
issues_df = input_df[["consumer_complaint_narrative"]].dropna()
issues_df.peek(n=5) # View an arbitrary five complaints

,consumer_complaint_narrative
1799560,"Thursday, XX/XX/XXXX, unauthorized charges wer..."
1800272,The credit reporting company is reporting inac...
1800409,In accordance with the Fair Credit Reporting a...
1800550,"I told the credit bureaus to "" investigate eve..."
1800818,Im writing in reference regarding XXXXXXXX XXX...


Downsample DataFrame to 10,000 records for model training.

In [8]:
# Choose 10,000 complaints randomly and store them in a column in a DataFrame
downsampled_issues_df = issues_df.sample(n=10000)

Generate the text embeddings

In [9]:
from bigframes.ml.llm import PaLM2TextEmbeddingGenerator

model = PaLM2TextEmbeddingGenerator() # No connection id needed

In [10]:
# Will take ~3 minutes to compute the embeddings
predicted_embeddings = model.predict(downsampled_issues_df)
# Notice the lists of numbers that are our text embeddings for each complaint
predicted_embeddings

,text_embedding,statistics,ml_embed_text_status,content
782,[ 2.78223325e-02 -1.71949025e-02 -5.01214415e-...,"{""token_count"":121,""truncated"":false}",,I 've sent multiple letters to this agency abo...
795,[ 1.39164589e-02 -5.08313216e-02 -4.53360938e-...,"{""token_count"":141,""truncated"":false}",,I receive social security XXXX funds in my XXX...
861,[ 7.80681521e-03 -3.23560014e-02 -6.76454604e-...,"{""token_count"":160,""truncated"":false}",,"Hello, My name is XXXX XXXX XXXX. I have a pro..."
1103,[ 4.18044440e-02 -4.28444222e-02 -6.26875088e-...,"{""token_count"":31,""truncated"":false}",,The debt occurred more than 7 years in the pas...
1241,[ 7.74183637e-03 -6.50701150e-02 -2.13856809e-...,"{""token_count"":23,""truncated"":false}",,UNAUTHORIZED CREDIT REPORTING NO ACCOUNT TO VA...
1729,[ 3.15782428e-02 -1.23979878e-02 -3.70296165e-...,"{""token_count"":382,""truncated"":false}",,XXXX on XXXX XX/XX/2021 I have Mr. Cooper mort...
2167,[ 9.87241510e-03 -1.81103535e-02 -4.17162031e-...,"{""token_count"":556,""truncated"":false}",,This is the third such complaint I have submit...
2219,[ 2.56749280e-02 -4.92163002e-02 -5.67202382e-...,"{""token_count"":196,""truncated"":false}",,Found and add online for a Prepaid Credit card...
2392,[ 2.34611966e-02 -4.74611111e-02 -3.59710641e-...,"{""token_count"":641,""truncated"":false}",,I am furnishing this complaint against Fed Loa...
2528,[ 1.90760177e-02 -4.90266569e-02 -5.60806654e-...,"{""token_count"":176,""truncated"":false}",,"Despite multiple written requests, the unverif..."


The model may have encountered errors while calculating embeddings for some rows. Filter out the errored rows before training the model. Alternatively, select these rows and retry the embeddings.

In [11]:
successful_rows = (
    (predicted_embeddings["ml_embed_text_status"] == "")
    # Series.str.len() gives the length of an array.
    # See: https://stackoverflow.com/a/41340543/101923
    & (predicted_embeddings["text_embedding"].str.len() != 0)
)
predicted_embeddings = predicted_embeddings[successful_rows]
predicted_embeddings


,text_embedding,statistics,ml_embed_text_status,content
782,[ 2.78223325e-02 -1.71949025e-02 -5.01214415e-...,"{""token_count"":121,""truncated"":false}",,I 've sent multiple letters to this agency abo...
795,[ 1.39164589e-02 -5.08313216e-02 -4.53360938e-...,"{""token_count"":141,""truncated"":false}",,I receive social security XXXX funds in my XXX...
861,[ 7.80681521e-03 -3.23560014e-02 -6.76454604e-...,"{""token_count"":160,""truncated"":false}",,"Hello, My name is XXXX XXXX XXXX. I have a pro..."
1103,[ 4.18044440e-02 -4.28444222e-02 -6.26875088e-...,"{""token_count"":31,""truncated"":false}",,The debt occurred more than 7 years in the pas...
1241,[ 7.74183637e-03 -6.50701150e-02 -2.13856809e-...,"{""token_count"":23,""truncated"":false}",,UNAUTHORIZED CREDIT REPORTING NO ACCOUNT TO VA...
1729,[ 3.15782428e-02 -1.23979878e-02 -3.70296165e-...,"{""token_count"":382,""truncated"":false}",,XXXX on XXXX XX/XX/2021 I have Mr. Cooper mort...
2167,[ 9.87241510e-03 -1.81103535e-02 -4.17162031e-...,"{""token_count"":556,""truncated"":false}",,This is the third such complaint I have submit...
2219,[ 2.56749280e-02 -4.92163002e-02 -5.67202382e-...,"{""token_count"":196,""truncated"":false}",,Found and add online for a Prepaid Credit card...
2392,[ 2.34611966e-02 -4.74611111e-02 -3.59710641e-...,"{""token_count"":641,""truncated"":false}",,I am furnishing this complaint against Fed Loa...
2528,[ 1.90760177e-02 -4.90266569e-02 -5.60806654e-...,"{""token_count"":176,""truncated"":false}",,"Despite multiple written requests, the unverif..."


We now have the complaints and their text embeddings as two columns in our predicted_embeddings DataFrame.

## Step 2: Create k-means model and predict clusters

In [12]:
from bigframes.ml.cluster import KMeans

cluster_model = KMeans(n_clusters=10) # We will divide our complaints into 10 groups

Perform KMeans clustering

In [13]:
# Use KMeans clustering to calculate our groups. Will take ~3 minutes.
cluster_model.fit(predicted_embeddings[["text_embedding"]])
clustered_result = cluster_model.predict(predicted_embeddings)
# Notice the CENTROID_ID column, which is the ID number of the group that
# each complaint belongs to.
clustered_result.peek(n=5)

,CENTROID_ID,NEAREST_CENTROIDS_DISTANCE,text_embedding,statistics,ml_embed_text_status,content
1094645,1,"[{'CENTROID_ID': 1, 'DISTANCE': 0.530282685572...",[ 7.32792774e-03 -7.59598315e-02 -4.49591577e-...,"{""token_count"":10,""truncated"":false}",,I do not have an account with this creditor
3372485,1,"[{'CENTROID_ID': 1, 'DISTANCE': 0.643931578310...",[-0.00161087 -0.04956109 -0.07371692 -0.057822...,"{""token_count"":10,""truncated"":false}",,Hard inquiries in my report that I do not reco...
2669308,1,"[{'CENTROID_ID': 1, 'DISTANCE': 0.599709344244...",[ 5.50241247e-02 -1.50039541e-02 -2.08624080e-...,"{""token_count"":100,""truncated"":false}",,I purchase {$25.00} for stock on the cash app ...
133816,1,"[{'CENTROID_ID': 1, 'DISTANCE': 0.618444281124...",[ 1.77251529e-02 -3.89547497e-02 -3.82236368e-...,"{""token_count"":100,""truncated"":false}",,BBVA fees I am in The Texas snow storm where I...
2697156,1,"[{'CENTROID_ID': 1, 'DISTANCE': 0.500398902102...",[-1.28429877e-02 -1.85956229e-02 -3.93197313e-...,"{""token_count"":1011,""truncated"":false}",,"After paying on my student loan for years, I o..."


Our DataFrame clustered_result now has an additional column that includes an ID from 1-10 (inclusive) indicating which semantically similar group they belong to.

## Step 3: Use Gemini to summarize complaint clusters

Build prompts - we will choose just two of our categories and prompt GeminiTextGenerator to identify their salient characteristics. The prompt is natural language in a python string.

In [14]:
# Using bigframes, with syntax identical to pandas,
# filter out the first and second groups
cluster_1_result = clustered_result[
    clustered_result["CENTROID_ID"] == 1
][["content"]]
cluster_1_result_pandas = cluster_1_result.head(5).to_pandas()

cluster_2_result = clustered_result[
    clustered_result["CENTROID_ID"] == 2
][["content"]]
cluster_2_result_pandas = cluster_2_result.head(5).to_pandas()

In [15]:
# Build plain-text prompts to send to PaLM 2. Use only 5 complaints from each group.
prompt1 = 'comment list 1:\n'
for i in range(5):
    prompt1 += str(i + 1) + '. ' + \
        cluster_1_result_pandas["content"].iloc[i] + '\n'

prompt2 = 'comment list 2:\n'
for i in range(5):
    prompt2 += str(i + 1) + '. ' + \
        cluster_2_result_pandas["content"].iloc[i] + '\n'

print(prompt1)
print(prompt2)

comment list 1:
1. This is the third such complaint I have submitted regarding the same type of issue over the past 12-18 months. 

On XX/XX/XXXX, my co-signed account was flagged by Navient as past due. The XXXX payment was mailed priority on XX/XX/XXXX and received by Navient on XX/XX/XXXX and delivered to " an individual '' per the post office. 
I called Navient on XX/XX/XXXX to talk to them about why my account was flagged since they received the payment long before the due date. The payment is sent via XXXX money orders under the same cover. The XXXX money order ( {$160.00} ) was cashed on XX/XX/XXXX per XXXX XXXX, the second money order ( {$250.00} ) which was sent in the same priority envelope and received the same time has not been cashed. 
When I called the customer service agent at Navient she told me that my account was past due and wanted me to send another payment. When I explained that they had received the payment she argued with me that if they received it, the payment 

In [16]:
# The plain English request we will make of PaLM 2
prompt = (
    "Please highlight the most obvious difference between "
    "the two lists of comments:\n" + prompt1 + prompt2
)
print(prompt)

Please highlight the most obvious difference between the two lists of comments:
comment list 1:
1. This is the third such complaint I have submitted regarding the same type of issue over the past 12-18 months. 

On XX/XX/XXXX, my co-signed account was flagged by Navient as past due. The XXXX payment was mailed priority on XX/XX/XXXX and received by Navient on XX/XX/XXXX and delivered to " an individual '' per the post office. 
I called Navient on XX/XX/XXXX to talk to them about why my account was flagged since they received the payment long before the due date. The payment is sent via XXXX money orders under the same cover. The XXXX money order ( {$160.00} ) was cashed on XX/XX/XXXX per XXXX XXXX, the second money order ( {$250.00} ) which was sent in the same priority envelope and received the same time has not been cashed. 
When I called the customer service agent at Navient she told me that my account was past due and wanted me to send another payment. When I explained that they ha

Get a response from Gemini by making a call to Vertex AI using our connection.

In [17]:
from bigframes.ml.llm import GeminiTextGenerator

q_a_model = GeminiTextGenerator()

In [18]:
# Make a DataFrame containing only a single row with our prompt for Gemini
df = bf.DataFrame({"prompt": [prompt]})

In [19]:
# Send the request for Gemini to generate a response to our prompt
major_difference = q_a_model.predict(df)
# PaLM 2's response is the only row in the dataframe result 
major_difference["ml_generate_text_llm_result"].iloc[0]

'The most obvious difference between the two lists of comments is the subject matter. The first list of comments is primarily focused on issues with financial institutions, such as Navient, Nelnet, PayPal, and Mr. Cooper. The second list of comments is primarily focused on issues with government agencies, such as the National Collegiate Trust, the USDA, and Ocwen.'

We now see GeminiTextGenerator's characterization of the different comment groups. Thanks for using BigQuery DataFrames!

# Summary and next steps

You've used the ML and LLM capabilities of BigQuery DataFrames to help analyze and understand a large dataset of unstructured feedback.

Learn more about BigQuery DataFrames in the [documentation](https://cloud.google.com/python/docs/reference/bigframes/latest) and find more sample notebooks in the [GitHub repo](https://github.com/googleapis/python-bigquery-dataframes/tree/main/notebooks).